<a href="https://colab.research.google.com/github/Murolando/Predictor_of_settelments/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказатель названия населенного пункта/ Predictor of the name of settlement

#### Идея проекта в том, чтобы сделать систему, которая при получении на вход название населенного пункта с ошибками, находила наиболее подходящий и возвращала его точное наименование 


---
#### The idea of the project is to make a system that, when receiving the name of a settlement with errors, found the most suitable one and returned its exact name



In [80]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re 
from collections import Counter

## Информация о данных / Structure of data

In [81]:
struc = pd.read_csv('/content/drive/MyDrive/ML Pr/Regions Of RF/structure.csv')

In [82]:
#print(struc)

## Основная часть / Main Part

In [83]:
df = pd.read_csv('/content/drive/MyDrive/ML Pr/Regions Of RF/data.csv')

In [84]:
df.head(5)

,id,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat
0,0,Орловская область,Болховский,Колонтаева,д,0,0,53.22.07,035.54.36,53.368611,35.910000,5.460442e+10,0,1
1,1,Республика Крым,Алушта,Пушкино,с,273,0,44.35.45,034.20.27,44.595833,34.340833,3.570300e+10,0,1
2,2,Липецкая область,Лев-Толстовский район,Барятино,с,7,1,53.15.46,039.30.14,53.262778,39.503889,4.263641e+10,0,1
3,3,Тверская область,Селижаровский район,Хилово,д,2,0,56.54.20,033.25.09,56.905556,33.419167,2.865043e+10,0,1
4,4,Томская область,Парабельский район,Басмасово,д,6,0,58.38.12,082.02.40,58.636667,82.044444,6.964444e+10,0,1


In [85]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155921 entries, 0 to 155920
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   id             155921 non-null  int64  
 1   region         155921 non-null  object 
 2   municipality   155921 non-null  object 
 3   settlement     155921 non-null  object 
 4   type           155921 non-null  object 
 5   population     155921 non-null  int64  
 6   children       155921 non-null  int64  
 7   latitude_dms   155921 non-null  object 
 8   longitude_dms  155921 non-null  object 
 9   latitude_dd    155921 non-null  float64
 10  longitude_dd   155921 non-null  float64
 11  oktmo          155920 non-null  float64
 12  dadata         155921 non-null  int64  
 13  rosstat        155921 non-null  int64  
dtypes: float64(3), int64(5), object(6)
memory usage: 16.7+ MB


### Модель будет пытаться ловить следующие ошибки:

▶ Вы допустили ошибку в названии населенного пункта

▶ Вы использовали лишние пробелы, забыли пробелы, неиспользовали тире, использовали лишние тире, использовали разные регистр и т.д

За основу я взял идею bag-of-words, однако вместо количества встречайний названия населенного пункта(далее нп), я буду использовать количество его населения, конечно в этой идеии есть и свои проблемы - множество туристически популярных городов, имеют небольшое население. По хорошему, мне стоило пробить каждый населеный пункт через word stat, но может быть как нибудь потом :D

Сначала оставим нужные нам столбцы

In [86]:
df_settlements = df[['settlement','type','population']]

Давайте посмотрим на данные 

In [87]:
df_settlements['population'].value_counts()

0        23441
1         6558
2         5867
3         4632
4         3904
         ...  
3902         1
14600        1
5348         1
38561        1
12566        1
Name: population, Length: 6172, dtype: int64

Как мы видим тут есть кучу записей, где количество жителей равно 0, давайте избавимся от них, т.к они не несут никакой полезной нагрузки 

In [88]:
df_settlements = df_settlements[df_settlements['population']!=0]

In [89]:
df_settlements['settlement'].value_counts()

Александровка    287
Ивановка         245
Михайловка       236
Никольское       210
Николаевка       202
                ... 
Пархачевская       1
Метеоритный        1
Порицы             1
Житново            1
Гостагаевская      1
Name: settlement, Length: 71708, dtype: int64

In [90]:
df_settlements.head()

,settlement,type,population
1,Пушкино,с,273
2,Барятино,с,7
3,Хилово,д,2
4,Басмасово,д,6
5,Каспа,с,375


In [91]:
df_settlements.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 132480 entries, 1 to 155920
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   settlement  132480 non-null  object
 1   type        132480 non-null  object
 2   population  132480 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 4.0+ MB


#### Давайте добавим функцию которая будет приводить название нашего пункта к универсальному виду, но сначала приведем исходные данные к нужному нам виду

1) Приведем все к одному регистру

In [92]:
df_settlements['settlement'] = df_settlements['settlement'].str.lower()

2) Уберем все пробелы,тире и числа = Оставим только буквы на русском 

In [93]:
df_settlements['settlement'] = df_settlements['settlement'].str.replace(r"[^а-я]", "", regex=True)

In [94]:
df_settlements = df_settlements[df_settlements['settlement']!='']

In [95]:
# Функция которая возвращает универсальное название нп
def universal(name_of_settlement):
  name_of_settlement = name_of_settlement.lower()

  name_of_settlement = re.sub("[^а-я]", "", name_of_settlement)
  # universal_name = ''
  # for i in name_of_settlement:
  #   if ord('a')<=ord(i)<=ord('z'):
  #     universal_name+=char(ord[i]-(ord(a)))
  return name_of_settlement

In [96]:
universal('СаНкт - ПитерБУргAasf')

'санктпитербург'

Как мы видим, тут есть множество повторов, уберем их(тем самым просто упростим задачу), однако если посмотреть на Москву, то она делится на муниципалитеты, и поэтому при объединение нам нужно соединить все популяции в одну строку 

In [97]:
print(df_settlements[df_settlements['settlement']=='Москва'])

Empty DataFrame
Columns: [settlement, type, population]
Index: []


In [98]:
df_settlements = df_settlements.groupby('settlement',as_index=False,observed = True).sum()

In [99]:
dfcitys= df_settlements['settlement']

### Сделаем из нашего датасета саму сумку слов, для этого используем словарь

In [100]:
COUNTS = {}
SUM_OF_ALL = 0
for i in range(0,len(df_settlements['settlement'])):
  COUNTS[df_settlements.iloc[i]['settlement']] = []
  COUNTS[df_settlements.iloc[i]['settlement']].append(df_settlements.iloc[i]['population'])
  SUM_OF_ALL += df_settlements.iloc[i]['population']
  # print(COUNTS)
  # print(type(int(a)))



Также посчитаем количество слов в котором встречается каждая из букв из алфавита, для теоремы байеса

In [108]:
alf = ('абвгдеёжзийклмнопрстуфхцчшщъыьэюя')
print(len(alf))
alf_in_words = {}
for j in alf:
  count = 0
  for i in range(0,len(df_settlements['settlement'])):
    if j in df_settlements.iloc[i]['settlement']:
      count+=1
  alf_in_words[j] = count
print(alf_in_words)

33
{'а': 42532, 'б': 13101, 'в': 33371, 'г': 10501, 'д': 11568, 'е': 32529, 'ё': 0, 'ж': 4407, 'з': 7401, 'и': 33660, 'й': 9417, 'к': 36641, 'л': 26588, 'м': 14588, 'н': 35303, 'о': 46480, 'п': 10178, 'р': 31872, 'с': 24079, 'т': 18857, 'у': 17278, 'ф': 1317, 'х': 7918, 'ц': 4115, 'ч': 7773, 'ш': 9991, 'щ': 1701, 'ъ': 189, 'ы': 10731, 'ь': 9291, 'э': 345, 'ю': 2480, 'я': 12223}


Я забыл назвать его с большой буквы, поэтому что не запускать это заново(а оно долго) я перепишу просто в новой ячейке

In [102]:
ALF_IN_WORDS = alf_in_words

In [109]:
print(ALF_IN_WORDS)

{'а': 42532, 'б': 13101, 'в': 33371, 'г': 10501, 'д': 11568, 'е': 32529, 'ё': 0, 'ж': 4407, 'з': 7401, 'и': 33660, 'й': 9417, 'к': 36641, 'л': 26588, 'м': 14588, 'н': 35303, 'о': 46480, 'п': 10178, 'р': 31872, 'с': 24079, 'т': 18857, 'у': 17278, 'ф': 1317, 'х': 7918, 'ц': 4115, 'ч': 7773, 'ш': 9991, 'щ': 1701, 'ъ': 189, 'ы': 10731, 'ь': 9291, 'э': 345, 'ю': 2480, 'я': 12223}


In [104]:
SUM_OF_ALL_IN = 0
for i in ALF_IN_WORDS:
  SUM_OF_ALL_IN += ALF_IN_WORDS[i]
print(SUM_OF_ALL_IN)

528425


### Предсказывать будем с помощью гениальной идеи будем каждое название  разбивать на буквы, и смотреть условную вероятность того, что буквы записанного слова, подходят под это слово

In [105]:
for words in COUNTS:
  COUNTS[words].append(Counter(words).most_common())
print(COUNTS['ардон'][0])

19412


In [106]:
print(type(COUNTS['ардон'][1][1]))

<class 'tuple'>


In [205]:
def pword(letters,word):
  variants = {}
  for words in COUNTS:
    candidate_word = words
    letters_of_candidate_word = COUNTS[words][1]
    pb = COUNTS[words][0]/SUM_OF_ALL
    pa_b = 1
    sum_of_p = 0
    kolv = 1
    if pb >0.5:
      print(pb)
    for letter in letters:
      let = letter[0]
      if let not in words:
        pa_b = 0
      pa = ALF_IN_WORDS[let]/SUM_OF_ALL_IN
      #pa = 1/33
      #print(let)
      pbayes = pb*pa_b/(pa)
      sum_of_p+=pbayes
      kolv+=1
      #print(sum_of_p)
    if sum_of_p >= 0.5:
      print(sum_of_p,pbayes,pa,pb,pa_b,let)
      variants[candidate_word] = sum_of_p/kolv
  return variants

In [206]:
def all_system(word):
  word = universal(word)
  letters = Counter(word).most_common()
  varients = pword(letters,word)
  print(varients)
      

### Тестировка

In [207]:
all_system('Владикавказ')

0.5315086668571793 0.0 0.014005771869234044 0.004288640165103792 0 з
3.579501252761463 0.0 0.014005771869234044 0.08386694196881235 0 з
{'владивосток': 0.06643858335714742, 'москва': 0.44743765659518286}


In [208]:
all_system('Санкт-питербург')

0.716086789536075 0.0 0.019872261910394096 0.007622788253948231 0 г
5.673744033799281 0.0 0.019872261910394096 0.03539888363072813 0 г
{'ростовнадону': 0.05508359919508269, 'санктпетербург': 0.43644184875379083}


In [209]:
all_system('Ногир')

0.942025656637439 0.14337096042014666 0.060315087287694566 0.008647431992261749 1 р
0.5298602125192621 0.0 0.060315087287694566 0.03539888363072813 0 р
{'нижнийновгород': 0.15700427610623982, 'санктпетербург': 0.08831003541987702}


In [210]:
all_system('Новосибирск')

2.7939717765614263 0.0 0.06934001987036949 0.08386694196881235 0 к
1.656722818234817 0.15807054355440642 0.06934001987036949 0.010960614630982647 1 к
{'москва': 0.31044130850682516, 'новосибирск': 0.18408031313720188}


In [211]:
all_system('Ростов')

0.9534722205221529 0.0 0.0631518190850168 0.08386694196881235 0 в
0.5468682392968707 0.0 0.0631518190850168 0.010960614630982647 0 в
0.7146481830351522 0.12070575898512462 0.0631518190850168 0.007622788253948231 1 в
0.5339607273917515 0.09018722274551742 0.0631518190850168 0.005695487174605028 1 в
{'москва': 0.15891203675369214, 'новосибирск': 0.09114470654947844, 'ростовнадону': 0.1191080305058587, 'саратов': 0.08899345456529191}


In [212]:
all_system('Владивосток')

0.7945613731847198 0.06184942221131988 0.06934001987036949 0.004288640165103792 1 к
0.7341667031923053 0.0 0.06934001987036949 0.006979941346138455 0 к
2.2814932210876036 0.0 0.06934001987036949 0.08386694196881235 0 к
{'владивосток': 0.07945613731847198, 'волгоград': 0.07341667031923052, 'москва': 0.22814932210876035}


# Выводы
👽 Все оказалось куда круче, чем я себе в голове представлял

Ну на самом деле все вышло не очень, такая модель никогда работать не будет, нужно было пересчитывать вероятности взависимости от порядка букв, или может еще какие еще решения, но мне понравилось, было весело, ну и я горжусь тем, что в популярные города он хотя бы добавляет их в варианты, что есть круто